<a href="https://colab.research.google.com/github/cosmaadrian/ml-environment/blob/master/DSM_Lab_FixMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSM Lab 6
## Semi-Supervised Learning with FixMatch

In this lab, we will explore the implementation of FixMatch, a method for performing semi-supervised learning.

We will perform our training on a fraction of CIFAR10, a dataset of natural images

You can find the original paper here: FixMatch: Simplifying Semi-Supervised Learning with Consistency and Confidence (https://arxiv.org/abs/2001.07685)



In [ ]:
import sys

import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torchvision.models import resnet18
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler

import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_epoch(model, dataloader, device, optimizer, criterion, epoch):
    model.train()

    total_train_loss = 0.0
    dataset_size = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader), colour='cyan', file=sys.stdout)
    for step, (images, labels) in bar:
        images = images.to(device)
        labels = labels.to(device)

        batch_size = images.shape[0]

        optimizer.zero_grad()
        pred = model(images)
        loss = criterion(pred, labels)

        loss.backward()
        optimizer.step()

        total_train_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = np.round(total_train_loss / dataset_size, 2)
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss)


    return epoch_loss

def valid_epoch(model, dataloader, device, criterion, epoch):
    model.eval()

    total_val_loss = 0.0
    dataset_size = 0

    correct = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader), colour='cyan', file=sys.stdout)
    for step, (images, labels) in bar:
        images = images.to(device)
        labels = labels.to(device)

        batch_size = images.shape[0]

        pred = model(images)
        loss = criterion(pred, labels)

        _, predicted = torch.max(pred, 1)
        correct += (predicted == labels).sum().item()

        total_val_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = np.round(total_val_loss / dataset_size, 2)

        accuracy = np.round(100 * correct / dataset_size, 2)

        bar.set_postfix(Epoch=epoch, Valid_Acc=accuracy, Valid_Loss=epoch_loss)

    return accuracy, epoch_loss

def run_training(model, trainloader, testloader, criterion, optimizer, num_epochs):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    top_accuracy = 0.0

    for epoch in range(num_epochs):

        train_loss = train_epoch(model, trainloader, device, optimizer, criterion, epoch)
        with torch.no_grad():
            val_accuracy, val_loss = valid_epoch(model, testloader, device, criterion, epoch)
            if val_accuracy > top_accuracy:
                print(f"Validation Accuracy Improved ({top_accuracy} ---> {val_accuracy})")
                top_accuracy = val_accuracy
        print()

In [ ]:
# Don't touch this for now.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Let's just use 10% of the data to make it harder
from collections import defaultdict
indices_per_class = defaultdict(list)
for i in range(len(cifar_trainset)):
  _, class_label = cifar_trainset[i]
  indices_per_class[class_label].append(i)

labeled_indices = []
unlabeled_indices = []
for class_name, indices in indices_per_class.items():
  labeled_indices.extend(indices[:int(0.1 * len(indices))]) # 10% labeled
  unlabeled_indices.extend(indices[int(0.1 * len(indices)):]) # 90% unlabeled

cifar_labeled_trainset = torch.utils.data.Subset(dataset = cifar_trainset, indices = labeled_indices)

cifar_labeled_trainloader = DataLoader(cifar_labeled_trainset, batch_size=64, shuffle=True)
cifar_testloader = DataLoader(cifar_testset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


# Baseline: Training just with the supervised data.

In [ ]:
learning_rate = 0.001
epochs = 10

model = torchvision.models.resnet18(pretrained = False) # let's initialize a ResNet18 from scratch and pretrain it ourselves
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=10, bias=True)

model.to(device)
criterion = nn.CrossEntropyLoss()

# Adam is an improved gradient descent algorithm
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
run_training(model, cifar_labeled_trainloader, cifar_testloader, criterion = criterion, optimizer = optimizer, num_epochs = epochs)

[INFO] Using GPU: Tesla T4

100%|██████████| 157/157 [00:04<00:00, 36.39it/s, Epoch=0, Valid_Acc=35.8, Valid_Loss=1.78]
Validation Accuracy Improved (0.0 ---> 35.83)

100%|██████████| 157/157 [00:04<00:00, 32.07it/s, Epoch=1, Valid_Acc=40.1, Valid_Loss=1.69]
Validation Accuracy Improved (35.83 ---> 40.11)

100%|██████████| 157/157 [00:04<00:00, 36.16it/s, Epoch=2, Valid_Acc=39.8, Valid_Loss=1.73]

100%|██████████| 157/157 [00:04<00:00, 36.41it/s, Epoch=3, Valid_Acc=40.4, Valid_Loss=1.85]
Validation Accuracy Improved (40.11 ---> 40.35)

100%|██████████| 157/157 [00:04<00:00, 31.85it/s, Epoch=4, Valid_Acc=40.9, Valid_Loss=1.96]
Validation Accuracy Improved (40.35 ---> 40.88)

100%|██████████| 157/157 [00:04<00:00, 32.65it/s, Epoch=5, Valid_Acc=40.5, Valid_Loss=2.07]

100%|██████████| 157/157 [00:04<00:00, 36.57it/s, Epoch=6, Valid_Acc=40.2, Valid_Loss=2.13]

100%|██████████| 157/157 [00:04<00:00, 31.86it/s, Epoch=7, Valid_Acc=40.5, Valid_Loss=2.26]

100%|██████████| 157/157 [00:04<00:00,

## Results suck. Let's make them better.

First, let's define a set of weak and strong augmentations.

In [ ]:
weak_transforms = transforms.Compose([
    # TODO add random horizontal flips
    # TODO add random crops (use padding=int(32*0.125))
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]) # weak transforms are just random shifting and flipping

strong_transforms = transforms.Compose([
    # TODO add random horizontal flips
    # TODO add random crops (use padding=int(32*0.125))
    # TODO add color jitter
    # TODO add random grayscale with some small probability
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class TransformFixMatch(object):
    def __init__(self, weak, strong):
        self.weak = weak
        self.strong = strong

    def __call__(self, x):
        weak = self.weak(x)
        strong = self.strong(x)

        return weak, strong

class CIFAR10SSL(datasets.CIFAR10):
    def __init__(self, root, indexs, train=True, transform=None, target_transform=None, download=False):
        super().__init__(root, train=train, transform=transform, target_transform=target_transform, download=download)
        if indexs is not None:
            self.data = self.data[indexs]
            self.targets = np.array(self.targets)[indexs]

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target


In [ ]:
# Adding the new transforms
cifar_labeled_trainset = CIFAR10SSL(root='./data', indexs = labeled_indices, train=True, transform = weak_transforms)
cifar_unlabeled_trainset = CIFAR10SSL(root='./data', indexs = unlabeled_indices, train=True, transform=TransformFixMatch(weak = weak_transforms, strong = strong_transforms))
cifar_testset = datasets.CIFAR10(root='./data', train=False, transform = val_transforms, download=False)

# One training batch contains 1/8 labeled data and 7/8 unlabeled data.
cifar_labeled_trainloader = DataLoader(cifar_labeled_trainset, batch_size=64, sampler = RandomSampler(cifar_labeled_trainset))
cifar_unlabeled_trainloader = DataLoader(cifar_unlabeled_trainset, batch_size=7 * 64, sampler = RandomSampler(cifar_unlabeled_trainset))

# Test set is all labeled
cifar_testloader = DataLoader(cifar_testset, batch_size=64, shuffle = False)

In [ ]:
def train_fixmatch_epoch(model, labeled_dataloader, unlabeled_dataloader, device, optimizer, epoch):
    criterion_labeled = nn.CrossEntropyLoss()
    criterion_unlabeled = nn.CrossEntropyLoss(reduction='none') # loss per example

    # TODO try out some other values here?
    threshold = 0.90 # predictions smaller than 90% confidence are filtered.

    model.train()

    total_train_loss = 0.0
    dataset_size = 0

    bar = tqdm(enumerate(unlabeled_dataloader), total=len(unlabeled_dataloader), colour='cyan', file=sys.stdout)

    labeled_iterator = iter(labeled_dataloader)

    epoch_loss = 0

    for step, (unlabeled_images, _) in bar:
        unlabeled_images_weak, unlabeled_images_strong = unlabeled_images

        unlabeled_images_weak = unlabeled_images_weak.to(device)
        unlabeled_images_strong = unlabeled_images_strong.to(device)
        try:
          labeled_images, labels = next(labeled_iterator)
        except StopIteration as e:
          labeled_iterator = iter(labeled_dataloader)
          labeled_images, labels = next(labeled_iterator)

        labeled_images = labeled_images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        pred_labeled = model(labeled_images)

        # get pseudo-labels, don't propagate gradients
        with torch.no_grad():
          pred_weak = model(unlabeled_images_weak)

          # get confidence as a probability
          pred_weak_confidence = torch.nn.functional.softmax(pred_weak, dim = -1)
          max_values, max_indices = torch.max(pred_weak_confidence, dim = -1)

          # filter out unconfident predictions
          fixmatch_mask = (max_values > threshold).float()

        # TODO other things to try out
        # add mixup between labeled data and unlabeled data
        # (interpolate labeled images with strongly augmented unlabeled images and between corresponding true labels and pseudo-labels)
        # mixup: BEYOND EMPIRICAL RISK MINIMIZATION https://arxiv.org/pdf/1710.09412

        # TODO (more complicated)
        # some pseudo-labels might be wrong and stay wrong throughout training
        # maybe figure out which ones are wrong by looking at training dynamics
        # Identifying Mislabeled Data using the Area Under the Margin Ranking https://arxiv.org/pdf/2001.10528
        # MarginMatch: Improving Semi-Supervised Learning with Pseudo-Margins https://arxiv.org/pdf/2308.09037

        pred_strong = model(unlabeled_images_strong)

        # loss for labeled images (nothing special, crossentropy between true labels and preds)
        loss_labeled = criterion_labeled(pred_labeled, labels)

        # loss for unlabeled: crossentropy between high-confidence pseudo-labels on weak augmentations and preds on strong augmentations
        # fixmatch_mask filters out unconfident pseudo-labels
        loss_consistency = criterion_unlabeled(pred_strong, max_indices) * fixmatch_mask
        loss_consistency = loss_consistency.mean()

        loss = loss_labeled + loss_consistency

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        bar.set_postfix(Epoch=epoch, LabeledLoss=loss_labeled.item(), ConsistencyLoss = loss_consistency.item(), FractionMasked=(1 - fixmatch_mask.float().mean()).item())

    return epoch_loss


def run_training_fixmatch(model, labeled_trainloader, unlabeled_trainloader, testloader, optimizer, num_epochs):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    top_accuracy = 0.0
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):

        train_loss = train_fixmatch_epoch(model, labeled_trainloader, unlabeled_trainloader, device, optimizer, epoch)
        with torch.no_grad():
            val_accuracy, val_loss = valid_epoch(model, testloader, device, criterion, epoch)
            if val_accuracy > top_accuracy:
                print(f"Validation Accuracy Improved ({top_accuracy} ---> {val_accuracy})")
                top_accuracy = val_accuracy
        print()

In [ ]:
learning_rate = 0.001
epochs = 40 # usually train for way longer.

model = torchvision.models.resnet18(pretrained = False) # let's initialize a ResNet18 from scratch and train it ourselves
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=10, bias=True)

model.to(device)
criterion = nn.CrossEntropyLoss()

# Adam is an improved gradient descent algorithm
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
run_training_fixmatch(
    model,
    labeled_trainloader = cifar_labeled_trainloader,
    unlabeled_trainloader = cifar_unlabeled_trainloader,
    testloader = cifar_testloader,
    optimizer = optimizer,
    num_epochs = epochs
)

[INFO] Using GPU: Tesla T4

100%|██████████| 157/157 [00:04<00:00, 34.99it/s, Epoch=0, Valid_Acc=41.6, Valid_Loss=1.62]
Validation Accuracy Improved (0.0 ---> 41.57)

100%|██████████| 157/157 [00:04<00:00, 35.45it/s, Epoch=1, Valid_Acc=44.3, Valid_Loss=1.54]
Validation Accuracy Improved (41.57 ---> 44.32)

100%|██████████| 157/157 [00:04<00:00, 34.31it/s, Epoch=2, Valid_Acc=47.2, Valid_Loss=1.47]
Validation Accuracy Improved (44.32 ---> 47.21)

100%|██████████| 157/157 [00:04<00:00, 32.80it/s, Epoch=3, Valid_Acc=49.9, Valid_Loss=1.4]
Validation Accuracy Improved (47.21 ---> 49.93)

100%|██████████| 157/157 [00:04<00:00, 32.04it/s, Epoch=4, Valid_Acc=52.4, Valid_Loss=1.34]
Validation Accuracy Improved (49.93 ---> 52.4)

100%|██████████| 157/157 [00:04<00:00, 34.40it/s, Epoch=5, Valid_Acc=49.5, Valid_Loss=1.47]

100%|██████████| 157/157 [00:04<00:00, 35.71it/s, Epoch=6, Valid_Acc=57.6, Valid_Loss=1.23]
Validation Accuracy Improved (52.4 ---> 57.65)

100%|██████████| 157/157 [00:04<00:00,

KeyboardInterrupt: ignored